In [1]:
!pip install darts

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from darts import TimeSeries
from darts.models import ARIMA, ExponentialSmoothing, Prophet, NBEATSModel, RNNModel
from darts.metrics import mae
from sklearn.model_selection import TimeSeriesSplit
import logging
import warnings
import torch

# Suppress cmdstanpy info messages
logging.getLogger("cmdstanpy").setLevel(logging.WARNING)
warnings.filterwarnings('ignore')

c:\Users\adana\anaconda3\envs\myenv\lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 13.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [3]:
# Load Data
train_processed_df = pd.read_csv('../data/final_train_cyclical.csv', index_col=0)
test_df = pd.read_csv('../data/final_test_cyclical.csv', index_col=0)

### 3. Modeling

#### Baseline Model
- [ ] **Baseline (Naive) Model**: Implement a simple baseline model, such as predicting the last known value or average to set a benchmark for evaluation.

#### Model Selection and Training
- [ ] **ARIMA / Exponential Smoothing**
- [ ] **LSTM/GRU**
- [ ] **Other Deep Learning**

#### Hyperparameter Tuning
- [ ] **Grid/Random Search**: Perform hyperparameter tuning using time-series cross-validation ().

#### Model Evaluation
- [ ] **Define Evaluation Metrics**: MAE
- [ ] **Evaluate on Validation Set**: Assess each model’s performance on the validation set and compare it with the baseline.
- [ ] **Residual Analysis**: Plot and analyze residuals to check for patterns or biases.

#### Forecasting
- [ ] **Make Predictions**: Generate predictions for the future time points provided in `test.csv`.

In [4]:
pollutants = ['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25']

In [5]:
# Function to train and evaluate using Darts with multiple models
models = {
    "ARIMA": ARIMA(p=1, d=1, q=1),
    "ExponentialSmoothing": ExponentialSmoothing(),
    "Prophet": Prophet(),
    "NBEATS": NBEATSModel(input_chunk_length=30, output_chunk_length=10, n_epochs=10, loss_fn=torch.nn.L1Loss(), random_state=42),
    "LSTM": RNNModel(model='LSTM', input_chunk_length=30, n_epochs=10, loss_fn=torch.nn.L1Loss(), random_state=42),
}
def train_and_evaluate_darts(pollutant, models):
    series = TimeSeries.from_dataframe(train_processed_df, time_col='id', value_cols=pollutant)
    
    maes = {model_name: [] for model_name in models.keys()}
    
    tscv = TimeSeriesSplit(n_splits=3)
        
    for train_index, test_index in tscv.split(series):
        train = series[:len(train_index)]  # Use the last index of train_index
        test = series[len(train_index): len(train_index) + len(test_index)]  # The test set starts after train set

        for model_name, model in models.items():
            
            # Fit the model
            model.fit(train)

            # Make predictions
            forecast = model.predict(len(test))

            # Evaluate the model
            score = mae(test, forecast)
            maes[model_name].append(score)
            
    # Calculate average MAE for each model
    avg_maes = {model_name: np.mean(scores) for model_name, scores in maes.items()}

    return avg_maes

In [ ]:
# Function to compute the average MAE across multiple pollutants
def compute_avg_mae_for_pollutants(pollutants, models):
    overall_maes = {model_name: [] for model_name in list(models.keys())}
    
    for pollutant in pollutants:
        avg_maes = train_and_evaluate_darts(pollutant, models)
        
        # Add each pollutant's average MAE to the overall list for each model
        for model_name, mae_value in avg_maes.items():
            overall_maes[model_name].append(mae_value)
    
    # Calculate the average MAE across all pollutants for each model
    final_avg_maes = {model_name: np.mean(scores) for model_name, scores in overall_maes.items()}

    return final_avg_maes

# Example usage:
overall_avg_maes = compute_avg_mae_for_pollutants(pollutants, models)

In [ ]:
# Train final models on full training data and make predictions on test set
for pollutant in pollutants:
    best_model = NBEATSModel(input_chunk_length=30, output_chunk_length=10, n_epochs=10, loss_fn=torch.nn.L1Loss(), random_state=42)
    series = TimeSeries.from_dataframe(train_processed_df, time_col='id', value_cols=pollutant)
    
    # Fit the model on the training series
    best_model.fit(series)
    
    # Make predictions for the length of the test set
    predictions = best_model.predict(len(test_df))
    
    # Convert predictions to a pandas DataFrame
    predictions_df = predictions.pd_dataframe()
    
    # Assign the predicted values to the corresponding column in test_df
    test_df[pollutant] = predictions_df[pollutant].values

In [19]:
predictions_df = test_df[['id', 'valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25']]
predictions_df['id'] = pd.to_datetime(predictions_df['id'])
predictions_df['id'] = predictions_df['id'].dt.strftime('%Y-%m-%d %H')

In [22]:
predictions_df

,id,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25
0,2024-09-03 23,27.562361,0.183950,37.864627,8.859465,4.602097
1,2024-09-04 00,26.069686,0.175708,38.507909,8.366501,4.487763
2,2024-09-04 01,25.776206,0.157750,39.418974,8.237877,4.727992
3,2024-09-04 02,32.470129,0.159022,37.578697,8.632615,5.534765
4,2024-09-04 03,39.412055,0.167750,35.631055,9.306403,5.914339
...,...,...,...,...,...,...
499,2024-09-24 18,38.941299,0.123977,5.295405,12.661563,8.890298
500,2024-09-24 19,39.258421,0.122299,4.423819,12.928077,8.710594
501,2024-09-24 20,38.561565,0.108837,5.037952,12.897201,7.993294
502,2024-09-24 21,36.624312,0.100032,5.656115,12.817968,7.810771


In [21]:
# Convert predictions to a DataFrame and save to CSV
predictions_df.to_csv('../data/submission.csv', index=False)

In [16]:
pd.read_csv('../data/sample_submission.csv')['id']

0      2024-09-03 23
1      2024-09-04 00
2      2024-09-04 01
3      2024-09-04 02
4      2024-09-04 03
           ...      
499    2024-09-24 18
500    2024-09-24 19
501    2024-09-24 20
502    2024-09-24 21
503    2024-09-24 22
Name: id, Length: 504, dtype: object

### LGBMRegressor

In [ ]:
!pip install tensorflow
!pip install optuna

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import TimeSeriesSplit


In [ ]:
train_data = pd.read_csv('final_train_cyclical.csv', index_col='Unnamed: 0', parse_dates=['id'])
test_data = pd.read_csv('final_test_cyclical.csv', index_col='Unnamed: 0', parse_dates=['id'])


train_data.set_index('id', inplace=True)
test_data.set_index('id', inplace=True)

# Define target columns
target_columns = ['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25']

# Define feature columns (exclude target variables and 'id')
features = [col for col in train_data.columns if col not in target_columns]

# Define the target and additional columns for lagging
target_columns = ['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25']
additional_columns = ['tavg', 'wspd', 'wdir']  # Adjust based on relevant features
columns_to_lag = target_columns + additional_columns
lag_periods = [1, 7, 30]  # Define lag periods

# Function to add lagged features
def add_lagged_features(data, columns, lags):
    for lag in lags:
        for col in columns:
            data[f"{col}_lag{lag}"] = data[col].shift(lag)
    return data

# Apply lagged features to both training and test datasets
train_data = add_lagged_features(train_data, columns_to_lag, lag_periods)

train_data.dropna(inplace=True)

# Dictionary to store models for each target variable
models = {}
for target in target_columns:
    # Initialize LightGBM model
    model = lgb.LGBMRegressor(n_estimators=1000, learning_rate=0.01)

    # Fit model on training data
    model.fit(train_data[features], train_data[target])

    # Store the model
    models[target] = model

# TimeSeriesSplit for cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Evaluate each model using cross-validation
mae_scores = {}
for target in target_columns:
    maes = []
    for train_idx, val_idx in tscv.split(train_data):
        X_train, X_val = train_data.iloc[train_idx][features], train_data.iloc[val_idx][features]
        y_train, y_val = train_data.iloc[train_idx][target], train_data.iloc[val_idx][target]

        # Train model
        model = lgb.LGBMRegressor(n_estimators=1000, learning_rate=0.01)
        model.fit(X_train, y_train)

        # Predict on validation set
        val_predictions = model.predict(X_val)

        # Calculate MAE and store it
        maes.append(mean_absolute_error(y_val, val_predictions))

    # Store average MAE for each target
    mae_scores[target] = np.mean(maes)
    print(f"Average MAE for {target}: {mae_scores[target]}")

# Calculate overall average MAE across all targets
average_mae = np.mean(list(mae_scores.values()))
print(f"Overall Average MAE: {average_mae}")

In [ ]:
# Dictionary to store predictions
predictions = {}

# Make predictions on the test set for each target variable
for target in target_columns:
    predictions[target] = models[target].predict(test_data[features])

# Convert predictions dictionary to a DataFrame
predictions_df = pd.DataFrame(predictions, index=test_data.index)
predictions_df.index = predictions_df.index.strftime('%Y-%m-%d %H')
predictions_df.reset_index(inplace=True)
predictions_df.rename(columns={'index': 'id'}, inplace=True)
predictions_df.head()

In [ ]:
# Save predictions to a CSV file for later analysis
predictions_df.to_csv("submission.csv", index=False)

### Ensemble method

In [ ]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
import optuna

In [ ]:

# Load data
train_data = pd.read_csv('final_train_cyclical.csv', index_col='Unnamed: 0', parse_dates=['id'])
test_data = pd.read_csv('final_test_cyclical.csv', index_col='Unnamed: 0', parse_dates=['id'])

train_data.set_index('id', inplace=True)
test_data.set_index('id', inplace=True)

# Define columns
target_columns = ['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25']
additional_columns = ['tavg', 'wspd', 'wdir']
columns_to_lag = target_columns + additional_columns
features = [col for col in train_data.columns if col not in target_columns]
lag_periods = [1, 7, 30]
rolling_windows = [3, 7, 30]

# Feature engineering function for lagged and rolling features
def add_features(data, columns, lag_periods, rolling_windows):
    for lag in lag_periods:
        for col in columns:
            data[f"{col}_lag{lag}"] = data[col].shift(lag)
    for window in rolling_windows:
        for col in columns:
            data[f"{col}_roll_mean_{window}"] = data[col].rolling(window).mean()
            data[f"{col}_roll_std_{window}"] = data[col].rolling(window).std()
    return data

# Add features to both train and test datasets
train_data = add_features(train_data, columns_to_lag, lag_periods, rolling_windows)
train_data.dropna(inplace=True)  # Drop NaNs in training data

# Hyperparameter tuning with Optuna
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 1500),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0)
    }
    model = lgb.LGBMRegressor(**params)
    tscv = TimeSeriesSplit(n_splits=5)
    maes = []
    for train_idx, val_idx in tscv.split(train_data):
        X_train, X_val = train_data.iloc[train_idx][features], train_data.iloc[val_idx][features]
        y_train, y_val = train_data.iloc[train_idx]['valeur_NO2'], train_data.iloc[val_idx]['valeur_NO2']
        model.fit(X_train, y_train)
        val_predictions = model.predict(X_val)
        maes.append(mean_absolute_error(y_val, val_predictions))
    return np.mean(maes)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)
best_params = study.best_params

# Train LightGBM and CatBoost models for each target with best parameters
models, catboost_models = {}, {}
for target in target_columns:
    lgb_model = lgb.LGBMRegressor(**best_params)
    lgb_model.fit(train_data[features], train_data[target])
    models[target] = lgb_model

    cat_model = CatBoostRegressor(iterations=1000, learning_rate=0.03, depth=6, silent=True)
    cat_model.fit(train_data[features], train_data[target])
    catboost_models[target] = cat_model

# Ensemble predictions by averaging LightGBM and CatBoost
def ensemble_predictions(test_data, models, catboost_models):
    predictions = {}
    for target in target_columns:
        lgb_pred = models[target].predict(test_data[features])
        cat_pred = catboost_models[target].predict(test_data[features])
        predictions[target] = (lgb_pred + cat_pred) / 2
    return pd.DataFrame(predictions, index=test_data.index)

predictions_df = ensemble_predictions(test_data, models, catboost_models)

# Format 'id' column and save to CSV
predictions_df.index = predictions_df.index.strftime('%Y-%m-%d %H')
predictions_df.reset_index(inplace=True)
predictions_df.rename(columns={'index': 'id'}, inplace=True)
predictions_df.to_csv("predictions_ensemble.csv", index=False)
